In [1]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

from collections import defaultdict
import pandas as pd
import numpy as np

from WA_state_API_functions import get_committee_member_data

# Load

In [2]:
engine = create_engine('postgresql://localhost:5432/wa_leg_raw')

In [3]:
vote_df = pd.read_sql_query('select * from "vote_api"',con=engine)

In [4]:
# bill_df = pd.read_sql_query('select * from "bill_api"',con=engine)

In [5]:
# bill_df['bill_unique'] = bill_df['bill_id'] + ' ' + bill_df['biennium']

In [6]:
# len(bill_df['bill_unique'].unique())

In [7]:
# bill_df.head()

In [8]:
vote_df['bill_unique'] = vote_df['bill_id'] + ' ' + vote_df['biennium']

In [9]:
len(vote_df['bill_unique'].unique())

19190

In [10]:
len(vote_df['voter_id'].unique())

530

In [11]:
committee_member_df = pd.read_sql_query('select * from "committee_member_api"',con=engine)

In [12]:
committee_member_df.head()

,acronym,agency,biennium,committee_name,district,first_name,id,last_name,long_name,name,party
0,ANDE,Senate,1993-94,Government Operations,42,Ann,6,Anderson,Senator Anderson,Ann Anderson,Republican
1,PELE,House,1995-96,Appropriations,25,Grant,1657,Pelesky,Representative Pelesky,Grant Pelesky,Republican
2,ELLI,House,1995-96,Education,1,Ian,1608,Elliot,Representative Elliot,Ian Elliot,Republican
3,MORR,House,1995-96,Finance,18,Betty,294,Morris,Representative Morris,Betty Morris,Democrat
4,HYME,House,1995-96,Health Care,40,Cheryl,1670,Hymes,Representative Hymes,Cheryl Hymes,Republican


# Start legislators_df

In [13]:
legislators_df = vote_df.loc[:, ['biennium', 'voter_id', 'voter_name', 'voting_agency']]

In [14]:
legislators_df.drop_duplicates(keep='first', inplace=True)

In [15]:
legislators_df.head()

,biennium,voter_id,voter_name,voting_agency
0,1991-92,7,Anderson,House
1,1991-92,11,Appelwick,House
2,1991-92,17,Ballard,House
3,1991-92,23,Basich,House
4,1991-92,32,Beck,House


In [16]:
committee_member_df_short = committee_member_df.loc[:, ['district', 'id', 'first_name', 'party']]

In [17]:
committee_member_df_short.columns = ['c_district', 'c_id', 'c_first_name', 'c_party']

In [18]:
committee_member_df_short['c_first_name'] = committee_member_df_short['c_first_name'].apply(lambda x: x.split()[0])

In [19]:
committee_member_df_short.drop_duplicates(keep='first', inplace=True)

In [43]:
legislators_df_merged = legislators_df.merge(committee_member_df_short, how='outer', left_on='voter_id', right_on='c_id')

In [44]:
legislators_df_merged.drop_duplicates(keep="first", inplace=True)

In [78]:
nulls = legislators_df_merged[legislators_df_merged['c_first_name'].isnull()]

In [87]:
nulls.loc[98:200, :]

,biennium,voter_id,voter_name,voting_agency,c_district,c_id,c_first_name,c_party
134,1991-92,186,Hine,House,NaN,NaN,NaN,NaN
141,1991-92,192,Holland,House,NaN,NaN,NaN,NaN
149,1991-92,205,Inslee,House,NaN,NaN,NaN,NaN
163,1991-92,214,"Johnson, R.",House,NaN,NaN,NaN,NaN
164,1993-94,214,"Johnson, R.",House,NaN,NaN,NaN,NaN
165,1991-92,219,Jones,House,NaN,NaN,NaN,NaN
166,1993-94,219,Jones,House,NaN,NaN,NaN,NaN
167,1991-92,226,"King, J.",House,NaN,NaN,NaN,NaN
168,1991-92,227,"King, R.",House,NaN,NaN,NaN,NaN
169,1993-94,227,King,House,NaN,NaN,NaN,NaN


In [46]:
manual = [{'name': 'Ann Anderson', 'voter_id': '7', 
  'district': '42', 'party': 'R'},
 {'name': 'Bob Basich', 'voter_id': '23', 
  'district': '19', 'party': 'D'},
 {'name': 'John Beck', 'voter_id': '32', 
  'district': '21', 'party': 'R'},
 {'name': 'Jennifer Belcher', 'voter_id': '34', 
  'district': '22', 'party': 'D'},
 {'name': 'John Betrozoff', 'voter_id': '40', 
  'district': '45', 'party': 'R'},
 {'name': 'Rose Bowman', 'voter_id': '45', 
  'district': '20', 'party': 'R'},
 {'name': 'Dennis Braddock', 'voter_id': '46', 
  'district': '42', 'party': 'D'},
 {'name': 'Lane Bray', 'voter_id': '47', 
  'district': '8', 'party': 'D'},
  {'name': 'Joanne Brekke', 'voter_id': '48', 
  'district': '32', 'party': 'D'},
  {'name': 'Art Broback', 'voter_id': '50', 
  'district': '28', 'party': 'R'},
  {'name': 'Jean-Marie Brough', 'voter_id': '54', 
  'district': '30', 'party': 'R'},
  {'name': 'Bill Brumsickle', 'voter_id': '63', 
  'district': '20', 'party': 'R'},
  {'name': 'Maria Cantwell', 'voter_id': '68', 
  'district': '44', 'party': 'D'},
  {'name': 'Gary Chandler', 'voter_id': '75', 
  'district': '13', 'party': 'R'},
  {'name': 'David Cooper', 'voter_id': '93', 
  'district': '18', 'party': 'D'},
  {'name': 'Bill Day', 'voter_id': '103', 
  'district': '3', 'party': 'D'},
  {'name': 'Dennis Dellwo', 'voter_id': '110', 
  'district': '3', 'party': 'D'},
  {'name': 'Randy Dorn', 'voter_id': '116', 
  'district': '2', 'party': 'D'},
  {'name': 'Brian Ebersole', 'voter_id': '121', 
  'district': '29', 'party': 'D'}, 
 {'name': 'Betty Edmondson', 'voter_id': '123', 
  'district': '14', 'party': 'R'},
  {'name': 'Roy Ferguson', 'voter_id': '132', 
  'district': '48', 'party': 'R'},
  {'name': 'Greg Fisher', 'voter_id': '135', 
  'district': '33', 'party': 'D'},
  {'name': 'Elmira Forner', 'voter_id': '140', 
  'district': '47', 'party': 'R'}, 
 {'name': 'Steve Fuhrman', 'voter_id': '146', 
  'district': '7', 'party': 'R'},
 {'name': 'Lorraine Hine', 'voter_id': '186', 
  'district': '33', 'party': 'D'},
 {'name': 'Bruce Holland', 'voter_id': '192', 
  'district': '47', 'party': 'R'},
 {'name': '', 'voter_id': '', 
  'district': '', 'party': ''},
 {'name': '', 'voter_id': '', 
  'district': '', 'party': ''},
 {'name': '', 'voter_id': '', 
  'district': '', 'party': ''},
 {'name': '', 'voter_id': '', 
  'district': '', 'party': ''},
 {'name': '', 'voter_id': '', 
  'district': '', 'party': ''},
 {'name': '', 'voter_id': '', 
  'district': '', 'party': ''},
 {'name': '', 'voter_id': '', 
  'district': '', 'party': ''},
 {'name': '', 'voter_id': '', 
  'district': '', 'party': ''},


In [47]:
manual_df = pd.DataFrame(manual)

In [48]:
manual_df.head()

,district,name,party,voter_id
0,42,Ann Anderson,R,7
1,19,Bob Basich,D,23
2,21,John Beck,R,32
3,22,Jennifer Belcher,D,34
4,45,John Betrozoff,R,40


In [49]:
manual_df['first_name'] = manual_df['name'].apply(lambda x: x.split()[0])

In [50]:
manual_df['name'] = manual_df['name'].apply(lambda x: x.split()[1])

In [51]:
manual_df.head()

,district,name,party,voter_id,first_name
0,42,Anderson,R,7,Ann
1,19,Basich,D,23,Bob
2,21,Beck,R,32,John
3,22,Belcher,D,34,Jennifer
4,45,Betrozoff,R,40,John


In [61]:
MERGED = legislators_df_merged.merge(manual_df, how='outer', on='voter_id')

In [62]:
MERGED.head()

,biennium,voter_id,voter_name,voting_agency,c_district,c_id,c_first_name,c_party,district,name,party,first_name
0,1991-92,7,Anderson,House,NaN,NaN,NaN,NaN,42,Anderson,R,Ann
1,1993-94,7,Anderson,House,NaN,NaN,NaN,NaN,42,Anderson,R,Ann
2,1995-96,7,"Anderson, C.",Senate,NaN,NaN,NaN,NaN,42,Anderson,R,Ann
3,1991-92,11,Appelwick,House,46,11,Marlin,Democrat,NaN,NaN,NaN,NaN
4,1993-94,11,Appelwick,House,46,11,Marlin,Democrat,NaN,NaN,NaN,NaN


In [75]:
def replace_null_with_first_name(x):
    if type(x['c_first_name']) == float:
        return x['first_name']
    else:
        return x['c_first_name']

In [76]:
MERGED['c_first_name'] = MERGED.apply(lambda x : replace_null_with_first_name(x),axis=1)

In [77]:
MERGED

,biennium,voter_id,voter_name,voting_agency,c_district,c_id,c_first_name,c_party,district,name,party,first_name
0,1991-92,7,Anderson,House,NaN,NaN,Ann,NaN,42,Anderson,R,Ann
1,1993-94,7,Anderson,House,NaN,NaN,Ann,NaN,42,Anderson,R,Ann
2,1995-96,7,"Anderson, C.",Senate,NaN,NaN,Ann,NaN,42,Anderson,R,Ann
3,1991-92,11,Appelwick,House,46,11,Marlin,Democrat,NaN,NaN,NaN,NaN
4,1993-94,11,Appelwick,House,46,11,Marlin,Democrat,NaN,NaN,NaN,NaN
5,1995-96,11,Appelwick,House,46,11,Marlin,Democrat,NaN,NaN,NaN,NaN
6,1997-98,11,Appelwick,House,46,11,Marlin,Democrat,NaN,NaN,NaN,NaN
7,1991-92,17,Ballard,House,12,17,Clyde,Republican,NaN,NaN,NaN,NaN
8,1993-94,17,Ballard,House,12,17,Clyde,Republican,NaN,NaN,NaN,NaN
9,1995-96,17,Ballard,House,12,17,Clyde,Republican,NaN,NaN,NaN,NaN


In [ ]:
  {'name': '', 'voter_id': '', 
  'district': '', 'party': ''},
  {'name': '', 'voter_id': '', 
  'district': '', 'party': ''}, 
 {'name': '', 'voter_id': '', 
  'district': '', 'party': ''},
  {'name': '', 'voter_id': '', 
  'district': '', 'party': ''},
  {'name': '', 'voter_id': '', 
  'district': '', 'party': ''},
  {'name': '', 'voter_id': '', 
  'district': '', 'party': ''}, 
 {'name': '', 'voter_id': '', 
  'district': '', 'party': ''},